In [ ]:
# Import packages
import numpy as np
import h5py
import glob
import re
import tensorflow as tf
from tensorflow.keras.models import Model

import matplotlib.pyplot as plt
from matplotlib import cm, colors
import matplotlib.ticker as ticker
from scipy.stats import pearsonr

np.set_printoptions(precision=3, suppress=True)

In [ ]:
# Define data loading
def extract_floats(string):
    return re.findall(r"[-+]?\d*\.\d+|\d+", string)

def data_load():
    density = 0.15
    files = glob.glob(f"../data/dataset_tumble_*_0.1_1000.h5") # IDeally this would be a test set to completely separate it from the training data
    inputs,outputs = [],[]
    for f in files:
        tumble = float(extract_floats(f)[0])
        with h5py.File(f, "r") as fin:
          count = 0
          for key in fin.keys():
              img = fin[key][:]
              img = img.reshape((img.shape[0], img.shape[1],1))
              shape = img.shape
              inputs.append(img)
              outputs.append(tumble)
              count+=1

    # Scramble the dataset
    order = np.arange(len(outputs)).astype(int)
    order = np.random.permutation(order)
    return np.array(inputs)[order],np.array(outputs)[order],shape

In [ ]:
x,y,shape = data_load()

In [ ]:
# Test train split
last = int(len(x)*0.2)
x_train, y_train = x[:-last], y[:-last]
x_val,y_val = x[-last:],y[-last:]

print("Size of training data: ", len(x_train))
print("Size of validation data: ", len(x_val))

In [ ]:
# Import model
name = "modelName"
model = tf.keras.models.load_model(f'../models/{name}.keras')


In [ ]:
# Select a layer and create partial model
layerNum = 0

print(model.layers[layerNum])
model_part = Model(
  inputs=model.layers[0].input,
  outputs=model.layers[layerNum].output)

intermed_output = model_part(x_train)
intermed_output.shape

# Get kernels and biases from original model
kernels, biases = model.layers[layerNum].get_weights()

In [ ]:
# Make prediction on validation set
prediction = model_part.predict(x_val)

In [ ]:
# Choosea frame number from validation set (ideally with percolation), then outputs each kernel in layer and it's output
example = 39

print(y_val[example])

for n in range(3):
    plt.subplot(2,3,n+1)
    plt.imshow(kernels[n].squeeze())
    plt.title(f"Kernel {n}")

    plt.subplot(2,3,n+4)
    plt.imshow(prediction[example][:,:,n])
    plt.title(f"It's output")

plt.text(-50,-70,"Third layer: conv_2d_1")

# plt.savefig("thirdLayerOutputNew.png")